In [ ]:
import numpy as np 
import pandas as pd
import re
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras import backend as K

from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

In [ ]:
train_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_train.csv',lineterminator='\n')
dev_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_valid.csv',lineterminator='\n')
test_df = pd.read_csv('../input/text-classification-dataset/hindi/hindi/hindi_test.csv',lineterminator='\n')

In [ ]:
train_df = train_df.drop(columns=['Unnamed: 0'])
dev_df = dev_df.drop(columns=['Unnamed: 0'])
test_df = test_df.drop(columns=['Unnamed: 0'])

In [ ]:
train_df.columns = ['text', 'label']
dev_df.columns = ['text', 'label']
test_df.columns = ['text', 'label']

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [ ]:
test_df.head()

In [ ]:
# Analysing train dataframe attributes 
print('* Size of dataframe: {}\n'.format(train_df.shape))
print('* Datatype of columns are:\n {}\n'.format(train_df.dtypes))
print('* Count of different categories:\n {}\n'.format(train_df['label'].value_counts()))
print('* Number of NaNs among text are: {}\n'.format(train_df['text'].isnull().sum()))

# Converting text to string
train_df['text'] = train_df['text'].astype(str)

# Removing NaNs
train_df = train_df.dropna(subset=['text'])
print('NaNs are removed from the dataframe. Number of NaNs can be confirmed to be {}. The size of dataframe has reduced to {}'.format(train_df['text'].isnull().sum(), train_df.shape))

# Analysing dev dataframe attributes 
print('* Size of dataframe: {}\n'.format(dev_df.shape))
print('* Datatype of columns are:\n {}\n'.format(dev_df.dtypes))
print('* Count of different categories:\n {}\n'.format(dev_df['label'].value_counts()))
print('* Number of NaNs among text are: {}\n'.format(dev_df['text'].isnull().sum())) 

# Converting text to string
dev_df['text'] = dev_df['text'].astype(str)

# Removing NaNs
dev_df = dev_df.dropna(subset=['text'])
print('NaNs are removed from the dataframe. Number of NaNs can be confirmed to be {}. The size of dataframe has reduced to {}'.format(dev_df['text'].isnull().sum(), dev_df.shape))

# Analysing test dataframe attributes 
print('* Size of dataframe: {}\n'.format(test_df.shape))
print('* Datatype of columns are:\n {}\n'.format(test_df.dtypes))
print('* Count of different categories:\n {}\n'.format(test_df['label'].value_counts()))
print('* Number of NaNs among text are: {}\n'.format(test_df['text'].isnull().sum()))

# Converting text to string
test_df['text'] = test_df['text'].astype(str)

# Removing NaNs
test_df = test_df.dropna(subset=['text'])
print('NaNs are removed from the dataframe. Number of NaNs can be confirmed to be {}. The size of dataframe has reduced to {}'.format(test_df['text'].isnull().sum(), test_df.shape))

In [ ]:
# Plotting label value counts
train_df.groupby('label').count().plot(kind='bar')
plt.show()

In [ ]:
# Plotting word lenghts of train text
train_word_length = [len(x) for x in train_df['text']]
plt.plot(train_word_length)
maxl = max(train_word_length)

In [ ]:
# Plotting label value counts
dev_df.groupby('label').count().plot(kind='bar')
plt.show()

In [ ]:
# Plotting word lenghts of train text
dev_word_length = [len(x) for x in dev_df['text']]
plt.plot(dev_word_length)

In [ ]:
# Plotting label value counts
test_df.groupby('label').count().plot(kind='bar')
plt.show()

In [ ]:
# Plotting word lenghts of train text
test_word_length = [len(x) for x in test_df['text']]
plt.plot(test_word_length)

# So this will be the maximum length of padding..

In [ ]:
tokenizer = Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
maxl

In [ ]:
max_length = 512

# As the length maximum length of the sequence is much, we will be selecting maximum length as 512.

In [ ]:
X_train = tokenizer.texts_to_sequences(train_df['text'].values)
X_train = pad_sequences(X_train, maxlen=max_length)
X_train.shape

In [ ]:
X_valid = tokenizer.texts_to_sequences(dev_df['text'].values)
X_valid = pad_sequences(X_valid, maxlen=max_length)
X_valid.shape

In [ ]:
X_test = tokenizer.texts_to_sequences(test_df['text'].values)
X_test = pad_sequences(X_test, maxlen=max_length)
X_test.shape

In [ ]:
Y_train = pd.get_dummies(train_df['label'],columns=train_df["label"]).values
Y_train.shape

In [ ]:
Y_valid = pd.get_dummies(dev_df['label'],columns=dev_df["label"]).values
Y_valid.shape

In [ ]:
Y_test = pd.get_dummies(test_df['label'],columns=test_df["label"]).values
Y_test.shape

# MODEL

In [ ]:
## Model parameters
embed_dim = 100

In [ ]:
model=Sequential()
model.add(Embedding(30000,embed_dim,input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, epochs=5, batch_size=64,validation_data=(X_valid, Y_valid),callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
model.save('model')

In [ ]:
# saving
with open('./tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

## Classification report

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

In [ ]:
print(classification_report(y_true, y_pred))

## Testing model with user input

In [ ]:
model = load_model("model")

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
labels_list = train_df['label'].unique()

In [ ]:
new_sentence = ['ఒకప్పుడు స్టార్‌ హీరోయిన్‌గా వెలుగువెలిగింది అందాల నటి మీనా. బాల నటిగా ఎంట్రీ ఇచ్చిన ఆమె తన అందచందాలతో, చక్కటి అభినయంతో చిత్ర పరిశ్రమలో తనకంటూ ప్రత్యేకమైన గుర్తింపు తెచ్చుకుంది. ఇటీవల అరుదైన దుబాయ్‌ గోల్డెన్ వీసాను కూడా అందుకుంది. అయితే కెరీర్‌ సరిగా లేని సమయంలో ప్రముఖ వ్యాపారవేత్త సాగర్‌ను ప్రేమించి పెళ్లి చేసుకుంది. వారికి ఒక పాప నైనిక ఉంది. ఆమె కూడా సినిమాల్లో నటిస్తోంది. కోలీవుడ్ స్టార్‌ హీరో విజయ్‌ నటించిన పోలీసోడు చిత్రంలో అతనికి కూతురుగా యాక్ట్‌ చేసి మెప్పించింది. కూతురు పుట్టిన తర్వాత కొంతకాలం వరకు సినిమాలు చేయలేదు మీనా.']
seq = tokenizer.texts_to_sequences(new_sentence)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = labels_list
print(pred, labels[np.argmax(pred)])

In [ ]:
new_sentence = ['ఐపీఎల్​ మ్యాచ్​లు గడుస్తున్నా కొద్దీ క్రికెట్ అభిమానుల్లో ఓ విషయంపై ఆందోళన పెరిగిపోతోంది. టీమ్ఇండియాకు మూలస్తంభాలుగా ఉన్న విరాట్ కోహ్లీ, రోహిత్ శర్మ.. పూర్తిగా లయ తప్పడం కలవరపాటుకు గురి చేస్తోంది. అయితే, ఈ సమయంలో వీరిద్దరికీ అండగా నిలవాలని మాజీలు కోరుతున్నారు. వీరి పేలవ ఫామ్ తాత్కాలికమేనని, త్వరలోనే తిరిగి విజృంభిస్తారని ధీమా వ్యక్తం చేస్తున్నారు.']
seq = tokenizer.texts_to_sequences(new_sentence)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = labels_list
print(pred, labels[np.argmax(pred)])

In [ ]:
new_sentence =['ఎన్టీఆర్ జిల్లా ఎ.కొండూరు మండలం కేస్యా తండాలో కిడ్నీ బాధితుల మరణాలు కొనసాగుతున్నాయి. ఒకే వారంలో ఇద్దరు మృత్యువాత పడగా.. గత 40 రోజులల్లో 8 మంది మృతిచెందారు. దీంతో స్థానికులు భయం గుప్పిట్లో బిక్కుబిక్కుమంటూ బతుకుతున్నారు.']
seq = tokenizer.texts_to_sequences(new_sentence)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = labels_list
print(pred, labels[np.argmax(pred)])

In [ ]:
new_sentence = ['వెండి ధరలు ఆదివారం స్వల్పంగా తగ్గాయి. ఆంధ్రప్రదేశ్​, తెలంగాణలో 10 గ్రాముల మేలిమి పుత్తడి ధర రూ.54వేల 100కుపైగా ఉంది. వెండి ధర కిలో రూ.70వేల దిగువన ఉంది. అంతర్జాతీయంగా క్రిప్టోకరెన్సీల్లో బిట్​ కాయిన్ విలువ పెరిగింది.']
seq = tokenizer.texts_to_sequences(new_sentence)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = labels_list
print(pred, labels[np.argmax(pred)])